# Twitter - Stocks Data Collection

## Objective

Get twitter information for the following Stocks
*['Walmart Inc.', 'Microsoft Corporation', 'The Home Depot','Alphabet Inc.', 'Apple Inc.', 'Wells Fargo','Chevron Corporation', 'The Coca-Cola Co','Exxon Mobil Corporation']*

1. We have used twitter api to get twitter information and also used Custom API to get older tweets relevant to the assets.
2. The data is collected from 1st January 2018 till 27th Feb 2019 [1 Year] for all 9 stocks
3. Then Vader NLTK Library is used to extract sentiment based on the tweet
4. Finally we collect sentiment data, titles for for all stocks hourly

In [3]:
#twitter Information
import pandas as pd
from os import listdir
from os.path import isfile, join

mypath = 'Twitter_Data_Cleaned/Stocks/'

# Mapping asset name to bring uniformity
stocks = [{'asset_name':'Wmt', 'name':'Walmart Inc.'},
{'asset_name':'MSFT', 'name': 'Microsoft Corporation'},
{'asset_name':'HD', 'name': 'The Home Depot'},
{'asset_name':'Google', 'name': 'Alphabet Inc.'},
{'asset_name':'AAPL', 'name': 'Apple Inc.'},
{'asset_name':'WFC', 'name': 'Wells Fargo'},
{'asset_name':'CVX', 'name': 'Chevron Corporation'},
{'asset_name':'KO', 'name': 'The Coca-Cola Co'},
{'asset_name':'XOM', 'name': 'Exxon Mobil Corporation'}]
reference_df = pd.DataFrame.from_records(stocks).rename({'asset_name':'stock',\
                                                         'name':'asset_name'}, axis=1)

complete_df=pd.DataFrame()

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file_name in onlyfiles:
    path_file_name = "Twitter_Data_Cleaned/Stocks/"+file_name
    if 'DS_Store' and '._' not in path_file_name:
        df = pd.read_csv(path_file_name)
        stock_name = file_name.split('.')[0]
        
        # Doing some cleaning
        mask = ((df['date'].str.len() == 16) & (~(df['date'].str.contains('[A-Za-z]',\
                regex=True, na=False))))
        df = df.loc[mask]
        if stock_name == 'Google': #Google data has float-type date
            df['date'] =  pd.to_datetime((df['date'].astype(float) - 25569)\
                            * 86400.0, unit='s')\
                            .astype('datetime64[s]')
        else:
            df['date'] =  pd.to_datetime(df['date']).dt.tz_localize(None)
        df['stock'] = stock_name
        complete_df=complete_df.append(df,ignore_index=True)
complete_df = complete_df.rename({'date':'created_utc'},axis=1)
complete_df = pd.merge(complete_df,reference_df,on=['stock'],how='inner')

complete_df.head(4)

,username,user_handle,created_utc,retweets,favorites,text,geological_location,mentions,hashtags,tweet_id,permalink,stock,asset_name
0,Microsoft Asia,MicrosoftASIA,2018-02-27 03:42:00,4,13,How #MSFT is contributing to #Taiwan 's ambiti...,NaN,NaN,#MSFT #Taiwan #energy,9.68E+17,https://twitter.com/MicrosoftASIA/status/96845...,MSFT,Microsoft Corporation
1,Microsoft Security,msftsecurity,2018-02-26 13:15:00,10,9,Creating a #cybersecurity plan is complex due ...,NaN,NaN,#cybersecurity #MSFT,9.68E+17,https://twitter.com/msftsecurity/status/968232...,MSFT,Microsoft Corporation
2,Pete Zerger,pzerger,2018-02-24 07:28:00,11,17,"Part 4 of my Microsft Cybersecurity Stack"" ser...",NaN,NaN,#msft #Azure #hyperv #IaaS,9.67E+17,https://twitter.com/pzerger/status/96742097561...,MSFT,Microsoft Corporation
3,Microsoft Azure,Azure,2018-02-22 08:15:00,17,26,#MSFT is structuring the #database for the age...,NaN,NaN,#MSFT #database #CosmosDB,9.67E+17,https://twitter.com/Azure/status/9667079552604...,MSFT,Microsoft Corporation


In [5]:
print('\x1b[1;31m Total number of tweets downloaded \x1b[0m',len(complete_df))

 Total number of tweets downloaded  468888


In [6]:
#Converting Titles to Sentiment Score
import nltk # be sure to have stopwords installed for this using nltk.download_shell()
import pandas as pd 
import string
import sqlite3
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
nltk.download('vader_lexicon')
#conn = sqlite3.connect("CMPT733.db")
#c = conn.cursor()
complete_df = complete_df[['asset_name','username','created_utc', 'retweets', \
                           'favorites', 'text','hashtags']]
sia = SIA()
sentiment = complete_df['text'].apply(lambda x : sia.polarity_scores(x))
complete_df=pd.concat([complete_df,sentiment.apply(pd.Series)],1)
complete_df.head(4)
#complete_df.to_sql('TBStocksTweets', con=conn, if_exists='append')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/abejju/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,asset_name,username,created_utc,retweets,favorites,text,hashtags,compound,neg,neu,pos
0,Microsoft Corporation,Microsoft Asia,2018-02-27 03:42:00,4,13,How #MSFT is contributing to #Taiwan 's ambiti...,#MSFT #Taiwan #energy,0.4767,0.0,0.807,0.193
1,Microsoft Corporation,Microsoft Security,2018-02-26 13:15:00,10,9,Creating a #cybersecurity plan is complex due ...,#cybersecurity #MSFT,0.4767,0.0,0.830,0.170
2,Microsoft Corporation,Pete Zerger,2018-02-24 07:28:00,11,17,"Part 4 of my Microsft Cybersecurity Stack"" ser...",#msft #Azure #hyperv #IaaS,0.3182,0.0,0.922,0.078
3,Microsoft Corporation,Microsoft Azure,2018-02-22 08:15:00,17,26,#MSFT is structuring the #database for the age...,#MSFT #database #CosmosDB,0.4019,0.0,0.886,0.114


In [8]:
import pandas as pd
from dateutil import parser
import datetime
import time
import sqlite3
conn = sqlite3.connect("CMPT733.db")
c = conn.cursor()

df = pd.read_sql("SELECT * FROM TBStocksTweets", conn)
df['created_utc'] = pd.to_datetime(df['created_utc'],infer_datetime_format=True)

ap = df.set_index('created_utc').groupby(['asset_name',pd.TimeGrouper('H')]).mean()
qp = df.set_index('created_utc').groupby(['asset_name',pd.TimeGrouper('H')]).count()
ap = ap.reset_index()
qp = qp.reset_index()
def row_to_list(x,col_name):
    return list(x[col_name])

fp = df.set_index('created_utc').groupby(['asset_name',pd.TimeGrouper('H')]).apply(lambda x: row_to_list(x,'text')).reset_index(name='text').dropna()
kp = df.set_index('created_utc').groupby(['asset_name',pd.TimeGrouper('H')]).apply(lambda x: row_to_list(x,'hashtags')).reset_index(name='hashtags').dropna()

ap = ap[['asset_name','created_utc','compound','neg','neu','pos']]
qp = qp[['asset_name','created_utc','retweets','favorites']]

result = pd.merge(ap,qp, how='inner',on=['asset_name','created_utc'])
result = pd.merge(result,fp, how='inner',on=['asset_name','created_utc'])
result = pd.merge(result,kp, how='inner',on=['asset_name','created_utc'])
result.head(4)
#result.to_json('twitter_stocks.json',orient='records',date_format='iso')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)


,asset_name,created_utc,compound,neg,neu,pos,retweets,favorites,text,hashtags
0,Alphabet Inc.,2017-12-31 16:00:00,0.124112,0.049767,0.863883,0.086350,60,60,[Check out the #NASDAQ or #NYSE trade of the w...,"[#NASDAQ #NYSE, #NASDAQ #NYSE #DayTrading #day..."
1,Alphabet Inc.,2017-12-31 17:00:00,0.373762,0.016172,0.834414,0.149448,29,29,"[As per the usual, February was another month ...","[None, None, None, #bizhour #SEOChat, None, No..."
2,Alphabet Inc.,2017-12-31 18:00:00,0.241308,0.065958,0.780875,0.153292,24,24,[Re-shared from 12/04/2017║Google now runs 3.0...,"[#energy, None, None, None, None, None, None, ..."
3,Alphabet Inc.,2017-12-31 19:00:00,0.201534,0.055719,0.793750,0.150500,32,32,[5-Star Methods For Increasing Positive Custom...,[#CustomerService #CX #Yelp #Facebook #Google ...


In [13]:
# Store the processed data in json
import pandas as pd
from dateutil import parser
import datetime
import time
result = pd.read_json('twitter_stocks.json')
result['created_utc'] = pd.to_datetime(result['created_utc'],infer_datetime_format=True)
analysis = result[['compound','asset_name']]
positive_df = analysis.rename({"compound":"positive_tweets"},axis='columns')
negative_df = analysis.rename({"compound":"negative_tweets"},axis='columns')

In [14]:
positive_df = positive_df[positive_df.positive_tweets > 0]
positive_df = positive_df.groupby(['asset_name']).count().reset_index()

negative_df = negative_df[negative_df.negative_tweets < 0]
negative_df = negative_df.groupby(['asset_name']).count().reset_index()
negative_df.head()
stats = pd.merge(positive_df,negative_df, how='inner',on=['asset_name','asset_name'])

In [16]:
import hvplot.pandas
import matplotlib.pyplot as plt

%matplotlib inline
stats.hvplot.bar(x="asset_name", y=["positive_tweets", "negative_tweets"],
                 stacked=True,rot=90,height=400, width=800,
                xlabel='Stocks',ylabel='tweets count', legend='top',
                title='Tweets Distribution for Stocks')

:Bars   [asset_name,Variable]   (value)